In [39]:
import os
import re

# SOURCE_FILE = "中国现代名中医医案精华（一）.txt"
SOURCE_FILE = '中国现代名中医医案精华.txt'

def preprocess(lines):
    '''
    @description: 预处理，删除无用行
    @param {list} lines: 每一行的内容
    @return {string}: 预处理后的内容
    '''

    # 移除仅包含数字的一行
    final_line = []
    for line in lines:
        line = line.strip()
        line += '\n'
        reg = re.compile(r'^\d+\n')
        province_reg = re.compile(r'.*(浙江|天津|山东|山西|江西|江苏|广东|贵州|福建|安徽).*')
        if re.match(reg, line):
           continue
        if re.match(province_reg, line):
            continue
        if line == '\n':
            continue
        final_line.append(line) 
    
    result = "".join(final_line)
    result = result.replace(" ", "") # 删除空格
    result = result.replace("　", "") # 删除全角空格
    result = result.replace("\t", "") # 删除制表符
    
    return result


def extract_from_author(content, author_name, ROOT_DIR, TARGET_DIR):
    '''
    @param content : 文本内容
    @param author_name : 作者名
    @param ROOT_DIR : 根目录
    @param TARGET_DIR : 目标目录
    @return {int} : 医案数量
    '''
    # 按照`一例` `两例`划分医案大类
    # 按照`例一` `例二`划分医案小类
    count = 0
    start_list = []
    record_list = []
    reg = re.compile(r"(?:.*)[一二三四五六七八九]例\n")
    for m in re.finditer(reg, content):
        if m.start() > 0:
            start_list.append(m.start())
    for m in range(len(start_list)):
        if m == len(start_list) - 1:
            record_list.append(content[start_list[m]:])
        else:
            record_list.append(content[start_list[m]:start_list[m+1]])

    count = 0
    for i in range(len(record_list)):
        first_line = record_list[i].split("\n")[0]
        # 删除`一例` `两例`，以获取疾病名称
        end_reg = re.compile(r"(?:,*)([一二三四五六七八九十]|十[一二三四五六七八九])例\n")

        # 获取疾病名称
        disease_name = re.sub(end_reg, "", first_line)
        if disease_name.find('治愈') != -1:
            disease_name = disease_name[disease_name.find('治愈')+2:]
        elif disease_name.find('治疗') != -1:
            disease_name = disease_name[disease_name.find('治疗')+2:]
        elif disease_name.find('治') != -1:
            disease_name = disease_name[disease_name.find('治')+1:]

        if disease_name.find('例') != -1:
            disease_name = disease_name[:disease_name.find('例')-1]

        if record_list[i].find('一例') != -1:
            with open(os.path.join(ROOT_DIR, TARGET_DIR, author_name + '-' + disease_name + ".txt"), "w", encoding="utf-8") as f:
                f.write(record_list[i])
                count += 1
            continue

        # 按照`例一` `例二`划分医案小类
        start_list_record = []
        record_list_record = []
        reg_record = re.compile(r"\n例[一二三四五六七八九].*\n")
        for m in re.finditer(reg_record, record_list[i]):
            if m.start() > 0:
                start_list_record.append(m.start())
        for m in range(len(start_list_record)):
            if m == len(start_list_record) - 1:
                record_list_record.append(record_list[i][start_list_record[m]:])
            else:
                record_list_record.append(record_list[i][start_list_record[m]:start_list_record[m+1]])
        
        for j in range(len(record_list_record)):
            file_name = author_name + '-' + disease_name
            if len(record_list_record) > 1:
                file_name += "-" + str(j+1) + ".txt"

            with open(os.path.join(ROOT_DIR, TARGET_DIR, file_name), "w", encoding="utf-8") as f:
                f.write(record_list_record[j])
                count += 1
    return count


def extract(content):
    '''
    @description: 抽取医案
    @param {string} content: 预处理后的内容
    @return {None}:
    '''
    ROOT_DIR = 'Record Collections MZY'
    TARGET_DIR = 'Raw Medical Record'
    if not os.path.exists(ROOT_DIR):
        os.mkdir(ROOT_DIR)
    if not os.path.exists(os.path.join(ROOT_DIR, TARGET_DIR)):
        os.mkdir(os.path.join(ROOT_DIR, TARGET_DIR))

    # 清空目标文件夹
    for file in os.listdir(os.path.join(ROOT_DIR, TARGET_DIR)):
        os.remove(os.path.join(ROOT_DIR, TARGET_DIR, file))

    author_reg = re.compile(r"\n.*医案\n")
    author_start_list = []
    author_record_list = []
    for m in re.finditer(author_reg, content):
        if m.start() > 0:
            author_start_list.append(m.start())
    for m in range(len(author_start_list)):
        if m == len(author_start_list) - 1:
            author_record_list.append(content[author_start_list[m]:])
        else:
            author_record_list.append(content[author_start_list[m]:author_start_list[m+1]])
    
    AUTHOR_DIR = 'Author Record'
    if not os.path.exists(os.path.join(ROOT_DIR, AUTHOR_DIR)):
        os.mkdir(os.path.join(ROOT_DIR, AUTHOR_DIR))
    for file in os.listdir(os.path.join(ROOT_DIR, AUTHOR_DIR)):
        os.remove(os.path.join(ROOT_DIR, AUTHOR_DIR, file))
    count = 0
    for i in range(len(author_record_list)):
        author_name = author_record_list[i].split("\n")[1]
        author_name = author_name.replace("医案", "")
        with open(os.path.join(ROOT_DIR, AUTHOR_DIR, author_name + ".txt"), "w", encoding="utf-8") as f:
            f.write(author_record_list[i])

        count += extract_from_author(author_record_list[i], author_name, ROOT_DIR, TARGET_DIR)

    print('医案抽取完成，共抽取{}个医案， 存放在{}目录下'.format(count, os.path.join(ROOT_DIR, TARGET_DIR)))


with open(SOURCE_FILE, "r", encoding="utf-8") as f:
    lines = f.readlines()
    content = preprocess(lines) # 预处理
    extract(content) # 抽取医案

医案抽取完成，共抽取559个医案， 存放在Record Collections MZY/Raw Medical Record目录下


In [49]:
# 生成带有标签的，用于导入的医案

def tag_generate(type, content, import_content):
    ''' 
    @param {string} type: 标签类型
    @param {string} content: 医案内容
    @param {string} import_content: 导入内容
    @return {string} import_content: 导入内容
    '''
    reg = re.compile(r"(辩证|辨证)[：:.。].*\n")
    if type == 'cure':
        reg = re.compile(r"(治法|治则)[：:.].*\n")
    for m in re.finditer(reg, content):
        dia_content = m.group()
        dia_content = dia_content[3:]
        dia_content = dia_content.strip()
        dia_content = dia_content.replace("。", "")
        dia_list = dia_content.split(",")
        for dia in dia_list:
            tmp_list = dia.split('，')
            for tmp in tmp_list:
                import_content += type + '-' + tmp + '\n'
    return import_content


def create_import_record():
    ROOT_DIR = 'Record Collections MZY'
    TARGET_DIR = 'Import Medical Record'
    if not os.path.exists(ROOT_DIR):
        os.mkdir(ROOT_DIR)
    if not os.path.exists(os.path.join(ROOT_DIR, TARGET_DIR)):
        os.mkdir(os.path.join(ROOT_DIR, TARGET_DIR))

    # 清空目标文件夹
    for file in os.listdir(os.path.join(ROOT_DIR, TARGET_DIR)):
        os.remove(os.path.join(ROOT_DIR, TARGET_DIR, file))

    for file in os.listdir(os.path.join(ROOT_DIR, 'Raw Medical Record')):
        with open(os.path.join(ROOT_DIR, 'Raw Medical Record', file), "r", encoding="utf-8") as f:
            origin_content = f.readlines()
            origin_content = ''.join(origin_content)

            import_content = ''
            import_content += '#book\n'
            import_content += '中国现代名中医医案精华\n'
            import_content += '#start\n'

            # 抽取'辩证'标签
            import_content = tag_generate('dia', origin_content, import_content)
            import_content = tag_generate('cure', origin_content, import_content)
            import_content += '#end\n'
            import_content += origin_content

            with open(os.path.join(ROOT_DIR, TARGET_DIR, file), "w", encoding="utf-8") as f:
                f.write(import_content)

create_import_record()